In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests, json, pandas as pd, numpy as np

In [2]:
url_comming_events = 'http://207.154.200.1/events/list/?tribe_paged=1&tribe_event_display=list'
url_past_events = 'http://207.154.200.1/events/list/?tribe_paged=1&tribe_event_display=past'

urls = [url_past_events, url_comming_events]

In [3]:
month_dict = {
    'Januar':1,
    'Februar':2,
    'März':3,
    'April':4,
    'Mai':5,
    'Juni':6,
    'Juli':7,
    'August':8,
    'September':9,
    'Oktober':10,
    'November':11,
    'Dezember':12,
}

In [4]:
year = datetime.now().year
saison_start = datetime(year, 4, 1)
saison_end = datetime(year, 10, 31)
relevant_weekdays = [5,6]
weekday_dict = {5:'Samstag',6:'Sonntag'}
holiday_api_url = f'https://feiertage-api.de/api/?jahr={year}'
country = 'BY'
file_name = f'NBDF_schleppkalender_pull_{datetime.now().strftime("%Y-%m-%d")}.xlsx'
holiday_obj = json.loads(requests.get(holiday_api_url).content)[country]
holiday_dict = dict()
for name, param in holiday_obj.items():
    holiday_str = param['datum']
    holiday = datetime.strptime(holiday_str,'%Y-%m-%d')
    hinweis = param['hinweis']
    if len(hinweis) == 0:
        holiday_dict[holiday] = name
holidays = list(holiday_dict.keys())
dates, calendar_weeks, weekdays = [], [], []
for month in range(1,13,1):
    for day in range(1,32,1):
        try:
            date = datetime(year, month, day)
            date_str = date.strftime('%d.%m.%Y')
            weekday = date.weekday()
            calendar_week = date.strftime('%V')
            if (weekday in relevant_weekdays or date in holidays) and date > saison_start and date < saison_end:
                if date in holidays:
                    weekday_str = holiday_dict[date]
                else:
                    weekday_str = weekday_dict[weekday]
                dates.append(date_str)
                calendar_weeks.append(calendar_week)
                weekdays.append(weekday_str)
        except:
            pass
        
windenfahrer, helfer = ['' for _ in range(len(dates))], ['' for _ in range(len(dates))]

In [5]:
year = datetime.now().year
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    event_list = soup.find_all('div',attrs={'class':'type-tribe_events'})
    for event in event_list:
        event_str = event.find('h3').text.strip()
        event_type = event_str.split(' ')[0]
        event_client = event_str.split(event_type)[1].strip()
        event_date_str = event.find('span').text.split('@')[0]
        month, day = event_date_str.strip().split(' ')
        month = month_dict[month]
        event_date = datetime(year,month,int(day))
        event_date_str = event_date.strftime('%d.%m.%Y')
        index = dates.index(event_date_str)
        if event_type.strip().lower() == 'windenfahrer':
            windenfahrer[index] = event_client
        elif event_type.strip().lower() == 'helfer':
            helfer[index] = event_client

In [6]:
df_dict = {
    'KW':calendar_weeks,
    'Wochentag':weekdays,
    'Datum':dates,
    'Windenfahrer':windenfahrer,
    'Helfer':helfer,
}

In [7]:
df = pd.DataFrame(df_dict)

In [8]:
with pd.ExcelWriter(file_name) as writer:
    df.to_excel(writer, sheet_name=str(year),index=False)

In [9]:
with open('schleppkalender_import.csv','w') as f:
    f.write('datum;event;\n')
    for idx, w in enumerate(windenfahrer):
        if len(w) > 0:
            datum = dates[idx]
            f.write(f'{datum};Windenfahrer {w};\n')
    for idx, h in enumerate(helfer):
        if len(h) > 0:
            datum = dates[idx]
            f.write(f'{datum};Helfer {h};\n')

In [10]:
print('All Done!')

All Done!
